### Methods for minimization with linear equality constraints
- Newton method requiring feasible start point
- Newton method allowing infeasible start point

### Methods for minimization with general inequality constraints
- Barrier method (iterated) using BFGS or Newton method

In [ ]:
import autograd.numpy as np
from autograd import grad as auto_grad, jacobian as auto_jacobian
import scipy

import matplotlib.pyplot as plt
import seaborn

In [ ]:
def solve_square_with_qr(A, b):
    """Solve system of equations with square (real values) matrix A: A * x = b"""""
    Q, R = np.linalg.qr(A)
    x = scipy.linalg.solve_triangular(R, Q.T @ b)
    return x


def solve_symm_with_chol(A, b):
    """Solve system of equations with symmetric (hermitian) matrix A: A * x = b"""""
    c, lower = scipy.linalg.cho_factor(A)
    x = scipy.linalg.cho_solve((c, lower), b)
    return x

In [ ]:
n0 = 1
def objective_fn0(x):
    y = - x[..., 0]**2
    return y

def eq_constraint_fn0(x):
    c0 = x[..., 0] - 1
    return np.array([c0])

grad_fn0 = auto_grad(objective_fn0)
hessian_fn0 = auto_jacobian(grad_fn0)
jacobian_eq_cnst0 = auto_jacobian(eq_constraint_fn0)

n1 = 2
def objective_fn1(x):
    y = x[..., 0]**2 + x[..., 1]**2
    return y

def eq_constraint_fn1(x):
#     c0 = x[..., 0]**2 + x[..., 1]**2 - 1
#     return np.array([c0])
#     c0 = x[..., 0]**2 - x[..., 1] - 1
#     return np.array([c0])
    c0 = 3*x[..., 0] - x[..., 1] - 1
    return np.array([c0])

grad_fn1 = auto_grad(objective_fn1)
hessian_fn1 = auto_jacobian(grad_fn1)
jacobian_eq_cnst1 = auto_jacobian(eq_constraint_fn1)


"""Objectives with inequality constraints"""

n1 = 1
def objective_fn2(x):
    y = - x[..., 0]**2
    return y

# Constraints as c <= 0
def constraint2_1(x):
    c = x[..., 0]**2 - 1
#     c = np.sqrt((x[..., 0] - 1)**2) - 0.5
    return np.array([c])

grad_fn2 = auto_grad(objective_fn2)
hessian_fn2 = auto_jacobian(grad_fn2)
constraints2 = [constraint2_1]
grad_constraints2 = [auto_grad(constraint) for constraint in constraints2]
hessian_constraints2 = [auto_jacobian(grad_cnst) for grad_cnst in grad_constraints2]


n1 = 2
def objective_fn3(x):
    y = x[..., 0]**2 + x[..., 1]**2
    return y


# Constraints as c <= 0

def constraint3_1(x):
    c = x[..., 0] + 1
    return np.array([c])

def constraint3_2(x):
    c = - x[..., 1] + 2
    return np.array([c])

grad_fn3 = auto_grad(objective_fn3)
hessian_fn3 = auto_jacobian(grad_fn3)
constraints3 = [constraint3_1, constraint3_2]
grad_constraints3 = [auto_grad(constraint) for constraint in constraints3]
hessian_constraints3 = [auto_jacobian(grad_cnst) for grad_cnst in grad_constraints3]

In [ ]:
def min_newton_method_eq_constraint_feasible(objective_fn, grad_fn, hessian_fn, constraint_fn, jacobian_cnst_fn, x0,
                                             alpha_min=1e-6, alpha_max=1e6,
                                             obj_tol=1e-15, cnst_tol=1e-15, max_iterations=10**6):
    """Minimize non-linear function with linear equality constraints.
    The initial point 'x0' has to be feasible."""

    assert np.abs(constraint_fn(x0)) < cnst_tol, \
        "The initial point has to be feasible ({} is above tolerance {})".format(
            np.abs(constraint_fn(x0)), cnst_tol)

    num_vars = x0.shape[0]
    num_cnst = constraint_fn(x0).shape[0]
    if x0.ndim == 1:
        x0 = x0[:, np.newaxis]

    x = np.array(x0)
    iteration = 0
    dobj = float("inf")
    obj_value = objective_fn(x[:, 0])
    while np.abs(dobj) > obj_tol and iteration < max_iterations:
        prev_obj_value = obj_value
        grad = grad_fn(x[:, 0])[:, np.newaxis]
        jacobian_cnst = jacobian_cnst_fn(x[:, 0])
        constraint = constraint_fn(x[:, 0])[:, np.newaxis]
        hessian = hessian_fn(x[:, 0])

        # Generate newton system
        A_upper = np.block([[hessian, jacobian_cnst.T]])
        A_lower = np.block([[jacobian_cnst, np.zeros((num_cnst, num_cnst))]])
        A = np.concatenate((A_upper, A_lower), axis=0)
        b = np.concatenate((-grad, np.zeros((num_cnst, 1))), axis=0)
        # Solve for step direction.
        # This is usually the main computational burden so should be
        # an efficient method depending on structure of A.
        y = np.linalg.solve(A, b)
        step_dir = y[:num_vars, :]
        # Lagrange variables
        nu = y[num_vars:, :]

        # Backtracking line-search
        alpha = alpha_max
        x_next = x
        dobj = float("inf")
        while dobj >= 0 and alpha >= alpha_min:
            step = alpha * step_dir
            x_next = x + step
            obj_value = objective_fn(x_next[:, 0])
            dobj = obj_value - prev_obj_value
            alpha *= 0.5
        iteration += 1

        if dobj >= 0:
            break
        x = x_next

    obj_value = objective_fn(x[:, 0])
    constraint_value = constraint_fn(x[:, 0])
    info = {
        "obj_value": obj_value,
        "constraint_value": constraint_value,
        "iteration": iteration,
        "max_iterations": max_iterations,
        "nu": nu,
    }
    x = x[:, 0]
    return x, info

In [ ]:
def min_newton_method_eq_constraint(objective_fn, grad_fn, hessian_fn,
                                    constraint_fn, jacobian_cnst_fn,
                                    x0, nu0=None,
                                    eps=1e-15, r_tol=1e-15, cnst_tol=1e-15, max_iterations=10**6):
    num_vars = x0.shape[0]
    num_cnst = constraint_fn(x0).shape[0]
    if nu0 is None:
        nu0 = np.zeros((num_cnst, 1))
    alpha = 0.25
    beta = 0.5
    if x0.ndim == 1:
        x0 = x0[:, np.newaxis]
    if nu0.ndim == 1:
        nu0 = nu0[:, np.newaxis]

    x = np.array(x0)
    nu = np.array(nu0)
    iteration = 0
    while iteration < max_iterations:
        grad = grad_fn(x[:, 0])[:, np.newaxis]
        jacobian_cnst = jacobian_cnst_fn(x[:, 0])
        constraint = constraint_fn(x[:, 0])[:, np.newaxis]
        hessian = hessian_fn(x[:, 0])

        # Generate newton system
        A_upper = np.block([[hessian, jacobian_cnst.T]])
        A_lower = np.block([[jacobian_cnst, np.zeros((num_cnst, num_cnst))]])
        A = np.concatenate((A_upper, A_lower), axis=0)
        b = np.concatenate((-grad + jacobian_cnst.T @ nu, -constraint), axis=0)
        # Solve for step direction.
        # This is usually the main computational burden so should be
        # an efficient method depending on structure of A.
        dy = np.linalg.solve(A, b)
        dx = dy[:num_vars, :]
        dnu = dy[num_vars:, :]
        print("dx:", dx, ", dnu:", dnu)

        r = lambda x, nu: np.concatenate((grad_fn(x[:, 0])[:, np.newaxis] + jacobian_cnst_fn(x[:, 0]).T @ nu, constraint_fn(x[:, 0])[:, np.newaxis]), axis=0)

        # Line search on newton decrement
        t = 1.
        r_value1 = r(x, nu)
        r_norm1 = np.linalg.norm(r_value1)
        while True:
            r_value2 = r(x + t * dx, nu + t * dnu)
            r_norm2 = np.linalg.norm(r_value2)
            if r_norm2 <= (1 - alpha * t) * r_norm1:
                break
            t *= beta
        x += t * dx
        nu += t * dnu
        iteration += 1

        if r_norm2 <= r_tol and np.abs(constraint_fn(x[:, 0])) <= cnst_tol:
            break
        if np.linalg.norm(t * dx) <= eps and np.linalg.norm(t * dy) <= eps:
            break


    obj_value = objective_fn(x[:, 0])
    constraint_value = constraint_fn(x[:, 0])
    info = {
        "obj_value": obj_value,
        "constraint_value": constraint_value,
        "r_value": r_value2,
        "iteration": iteration,
        "max_iterations": max_iterations,
        "t": t,
        "nu": nu[:, 0],
    }
    x = x[:, 0]
    return x, info

In [ ]:
# Minimize function #0 with newton method
x0 = np.array([5.])
max_iterations = 10
x_min, info = min_newton_method_eq_constraint(objective_fn0, grad_fn0, hessian_fn0,
                                              eq_constraint_fn0, jacobian_eq_cnst0, x0,
                                              max_iterations=max_iterations)
print("minimum x: {}".format(x_min))
print("objective value: {}".format(info["obj_value"]))
print("constraint value: {}".format(info["constraint_value"]))
# print("r_value: {}".format(info["r_value"]))
print("iteration: {}".format(info["iteration"]))

# Plot function and found minimum
plt.figure()
x = np.linspace(-np.abs(x_min), +np.abs(x_min), 100)[:, np.newaxis]
plt.plot(x, objective_fn0(x))
plt.plot([x_min], [objective_fn0(x_min)], 'rx')
plt.show()

In [ ]:
# Minimize function #1 with newton method
x0 = np.array([2., 5.])
x0 = np.array([2., 5000.])
# x0 = np.array([2., 0.])
print("Constraint value of initial point:", eq_constraint_fn1(x0))
max_iterations = 10
x_min, info = min_newton_method_eq_constraint(
    objective_fn1, grad_fn1, hessian_fn1,
    eq_constraint_fn1, jacobian_eq_cnst1, x0,
    max_iterations=max_iterations)
print("minimum x: {}".format(x_min))
print("objective value: {}".format(info["obj_value"]))
print("constraint value: {}".format(info["constraint_value"]))
# print("r_value: {}".format(info["r_value"]))
print("iteration: {}".format(info["iteration"]))

# Plot function and found minimum
plt.figure()
x = np.linspace(-np.abs(x_min[0]), +np.abs(x_min[0]), 100)[:, np.newaxis]
plt.plot(x, [objective_fn1(np.array([v, x_min[1]])) for v in x])
plt.plot([x_min[0]], [objective_fn1(x_min)], 'rx')
plt.show()

plt.figure()
x = np.linspace(-np.abs(x_min[1]), +np.abs(x_min[1]), 100)[:, np.newaxis]
plt.plot(x, [objective_fn1(np.array([x_min[0], v])) for v in x])
plt.plot([x_min[1]], [objective_fn1(x_min)], 'rx')
plt.show()

In [ ]:
# Minimize function #1 with newton method starting from feasible point
x0 = np.array([2., 5.])
max_iterations = 100
x_min, info = min_newton_method_eq_constraint_feasible(
    objective_fn1, grad_fn1, hessian_fn1,
    eq_constraint_fn1, jacobian_eq_cnst1,
    x0, max_iterations=max_iterations)
print("minimum x: {}".format(x_min))
print("objective value: {}".format(info["obj_value"]))
print("constraint value: {}".format(info["constraint_value"]))
print("iteration: {}".format(info["iteration"]))

# Plot function and found minimum
plt.figure()
x = np.linspace(-np.abs(x_min[0]), +np.abs(x_min[0]), 100)[:, np.newaxis]
plt.plot(x, [objective_fn1(np.array([v, x_min[1]])) for v in x])
plt.plot([x_min[0]], [objective_fn1(x_min)], 'rx')
plt.show()

plt.figure()
x = np.linspace(-np.abs(x_min[1]), +np.abs(x_min[1]), 100)[:, np.newaxis]
plt.plot(x, [objective_fn1(np.array([x_min[0], v])) for v in x])
plt.plot([x_min[1]], [objective_fn1(x_min)], 'rx')
plt.show()

In [ ]:
def min_newton_method(objective_fn, grad_fn, hessian_fn, x0,
                      t_min=1e-6, eps=1e-15, obj_tol=1e-15, max_iterations=10**6,
                      verbose=False):
    x = np.array(x0)
    t = 1.
    iteration = 0
    dobj = float("inf")
    obj_value = objective_fn(x)
    print(obj_value, iteration, dobj, obj_tol)
    while np.abs(dobj) > obj_tol and iteration < max_iterations:
        if verbose:
            print("iteration:", iteration)
            print("  x:", x)
        prev_obj_value = obj_value
        grad = grad_fn(x)
        hessian = hessian_fn(x)
        if verbose:
            print("  grad:", grad)
            print("  hessian:", hessian)

        # Compute step direction (+ simple method for Hessian modification)
        hessian_mod_factor = 1
        hessian_mod = hessian
        positive_definite = False
        while not positive_definite:
            try:
                # Cholesky factorization will fail if hessian not positive definite
                step = solve_symm_with_chol(hessian_mod, -grad)
                positive_definite = True
            except np.linalg.LinAlgError as exc:
                # In that case we add a multiple of the identity matrix
                hessian_mod = hessian + hessian_mod_factor * np.eye(hessian.shape[0])
                # And increase factor if still not positive definite
                hessian_mod_factor *= 2
        if verbose:
            print("  hessian_mod:", hessian_mod)
            print("  step:", step)

        newton_decrement_square = - grad.T @ step
        print(grad, step, newton_decrement_square)
        if newton_decrement_square * 0.5 < eps:
            if verbose:
                print("Newton decrement below threshold")
            break

        # Backtracking line search
        t = 1.
        x_next = x
        dobj = float("inf")
        while dobj > 0 and t >= t_min:
            x_next = x + t * step
            obj_value = objective_fn(x_next)
            dobj = obj_value - prev_obj_value
            t *= 0.5
        if verbose:
            print("Line search stopped with t={}, x_next={}, dobj={}, obj_value={}".format(
                t, x_next, dobj, obj_value))
        x = x_next
        iteration += 1
    info = {
        "obj_value": obj_value,
        "dobj": dobj,
        "obj_tol": obj_tol,
        "iteration": iteration,
        "max_iterations": max_iterations,
        "grad": grad,
        "hessian": hessian,
        "hessian_mod": hessian_mod,
        "t": t,
        "step": step,
    }
    return x, info

In [ ]:
def min_bfgs(objective_fn, grad_fn, x0,
             t_min=1e-6, eps=1e-15, obj_tol=1e-15, beta=0.5, c=0.0001,
             max_iterations=10**6, verbose=False):
    x = np.array(x0)
    # Initialize inverse Hessian approximation
    B_inv = np.eye(x.shape[-1])
    t = None
    step = None
    iteration = 0
    dobj = float("inf")
    obj_value = objective_fn(x)
    while np.abs(dobj) > obj_tol and iteration < max_iterations:
        if verbose:
            print("iteration:", iteration)
        prev_obj_value = obj_value
        grad = grad_fn(x)
        if verbose:
            print("  grad:", grad)
            print("  B_inv:", B_inv)

        # Simple method for Hessian modification (faster methods available)
        hessian_mod_factor = 1
        B_inv_mod = B_inv
        positive_definite = False
        while not positive_definite:
            try:
                # Cholesky factorization will fail if hessian not positive definite
                scipy.linalg.cho_factor(B_inv_mod)
                positive_definite = True
            except np.linalg.LinAlgError as exc:
                # In that case we add a multiple of the identity matrix
                B_inv_mod = B_inv + hessian_mod_factor * np.eye(B_inv.shape[0])
                # And increase factor if still not positive definite
                hessian_mod_factor *= 2

        step_dir = -B_inv_mod @ grad
        if verbose:
            print("  step_dir:", step_dir)

        newton_decrement_square = - grad.T @ step_dir
        if newton_decrement_square * 0.5 < eps:
            if verbose:
                print("Newton decrement below threshold")
            break

        t = 1.
        x_next = x
        grad_norm = np.linalg.norm(grad)
        while t >= t_min:
            step = t * step_dir
            x_next = x + step
            obj_value = objective_fn(x_next)
            dobj = obj_value - prev_obj_value
            # Armijo-Goldstein inequality
            if dobj <= - c * t * grad_norm:
                break
            t *= beta
        x = x_next
        step = step[..., np.newaxis]

        # Update inverse Hessian approximation
        y = (grad_fn(x) - grad)[..., np.newaxis]
        B_inv_step1_nom = (step.T @ y + y.T @ B_inv @ y) * (step @ step.T)
        B_inv_step1_denom = (step.T @ y) ** 2
        B_inv_step1 = B_inv_step1_nom / B_inv_step1_denom
        B_inv_step2 = - (B_inv @ y @ step.T + step @ y.T @ B_inv) / (step.T @ y)
        B_inv += B_inv_step1 + B_inv_step2

        iteration += 1
    info = {
        "obj_value": obj_value,
        "dobj": dobj,
        "obj_tol": obj_tol,
        "iteration": iteration,
        "max_iterations": max_iterations,
        "grad": grad,
        "t": t,
        "step_dir": step_dir,
        "step": step,
        "B_inv": B_inv,
    }
    return x, info

In [ ]:
def min_barrier_method(obj_fn, grad_fn, hessian_fn,
                       constraints, grad_constraints, hessian_constraints,
                       x0, max_iterations=1000, t0=0.01, t_factor=10., eps=1e-12, cnst_tol=1e-300,
                       use_bfgs=True, verbose=False):

    if len(constraints) == 0:
        if use_bfgs:
            return min_bfgs(obj_fn, grad_fn, x0, max_iterations=max_iterations)
        else:
            return min_newton_method(obj_fn, grad_fn, hessian_fn, x0, max_iterations=max_iterations)

    for i, constraint_fn in enumerate(constraints):
        cnst = constraint_fn(x0)
        assert cnst <= -cnst_tol, \
            "The initial point has to be feasible for constraint {} ({} is above tolerance {})".format(
                i, np.abs(cnst), cnst_tol)

    def fun_with_barrier(x, t):
        barrier = 0
        for constraint_fn in constraints:
            cnst = constraint_fn(x)
            if cnst >= -cnst_tol:
                return float("inf")
            barrier += - np.log(-cnst)
        y = obj_fn(x) + 1 / t * barrier
        return y

    def grad_with_barrier(x, t):
        barrier_grad = 0
        for constraint_fn, grad_constraint_fn in zip(constraints, grad_constraints):
            cnst = constraint_fn(x)
            grad_cnst = grad_constraint_fn(x)
            barrier_grad += - (1 / cnst) * grad_cnst
        grad = grad_fn(x) + 1 / t * barrier_grad
        return grad

    if not use_bfgs:
        def hessian_with_barrier(x, t):
            barrier_hessian = 0
            for constraint_fn, grad_constraint_fn, hessian_constraint_fn \
            in zip(constraints, grad_constraints, hessian_constraints):
                cnst = constraint_fn(x)
                grad_cnst = grad_constraint_fn(x)
                hessian_cnst = hessian_constraint_fn(x)
                barrier_hessian += (1 / cnst**2) * grad_cnst[..., np.newaxis] @ grad_cnst[..., np.newaxis].T
                barrier_hessian += - (1 / cnst**2) * hessian_cnst
            hessian = hessian_fn(x) + 1 / t * barrier_hessian
            return hessian

    x = x0
    t = t0
    m = len(constraints)
    outer_iteration = 0
    info = {}

    while m / t >= eps:
        if use_bfgs:
            x_min, new_info = min_bfgs(
                lambda x: fun_with_barrier(x, t),
                lambda x: grad_with_barrier(x, t),
                x, max_iterations=max_iterations, verbose=verbose)
        else:
            x_min, new_info = min_newton_method(
                lambda x: fun_with_barrier(x, t),
                lambda x: grad_with_barrier(x, t),
                lambda x: hessian_with_barrier(x, t),
                x, max_iterations=max_iterations, verbose=verbose)

        if not np.isfinite(new_info['obj_value']):
            if verbose:
                print("Objective value diverged. Stopping.")
            break
        if verbose:
            print("x_min:", x_min)
        x = x_min
        info = new_info
        t *= t_factor
        outer_iteration += 1

    info["outer_iteration"] = outer_iteration
    info["t_barrier"] = t
    return x, info

In [ ]:
x0 = np.array([-200., 5.])
max_iterations = 100
x_min, info = min_barrier_method(
    objective_fn3, grad_fn3, hessian_fn3,
    constraints3, grad_constraints3, hessian_constraints3,
    x0, max_iterations=max_iterations, verbose=False)

print("minimum x: {}".format(x_min))
print("objective value: {}".format(info["obj_value"]))
print("constraint values: {}".format([constraint_fn(x_min) for constraint_fn in constraints2]))
print("iteration: {}".format(info["iteration"]))
print("outer iterations: {}".format(info["outer_iteration"]))

# Plot function and found minimum
plt.figure()
x = np.linspace(-np.abs(x_min[0]), +np.abs(x_min[0]), 100)[:, np.newaxis]
plt.plot(x, [objective_fn3(np.array([v, x_min[1]])) for v in x])
plt.plot([x_min[0]], [objective_fn3(x_min)], 'rx')
plt.show()

plt.figure()
x = np.linspace(-np.abs(x_min[1]), +np.abs(x_min[1]), 100)[:, np.newaxis]
plt.plot(x, [objective_fn3(np.array([x_min[0], v])) for v in x])
plt.plot([x_min[1]], [objective_fn3(x_min)], 'rx')
plt.show()

In [ ]:
x0 = np.array([0.5])
max_iterations = 100
x_min, info = min_barrier_method(
    objective_fn2, grad_fn2, hessian_fn2,
    constraints2, grad_constraints2, hessian_constraints2,
    x0, t0=10, max_iterations=max_iterations, verbose=False)

print("minimum x: {}".format(x_min))
print("objective value: {}".format(info["obj_value"]))
print("constraint values: {}".format([constraint_fn(x_min) for constraint_fn in constraints2]))
print("iteration: {}".format(info["iteration"]))

# Plot function and found minimum
plt.figure()
x = np.linspace(-np.abs(x_min[0]), +np.abs(x_min[0]), 100)[:, np.newaxis]
plt.plot(x, objective_fn2(x))
plt.plot([x_min[0]], [objective_fn2(x_min)], 'rx')
plt.show()